In [1]:
import org.apache.spark.sql.functions.input_file_name
import scala.sys.process.Process
import sys.process._
import java.io.File;
import scala.Byte
import scala.util.Random
import java.nio.file.{Files, Paths}
import java.nio.ByteBuffer
import java.util.Arrays 
import java.nio.ByteOrder
import java.io.RandomAccessFile
import java.io.PrintWriter
import java.io.FileWriter
import scala.collection.mutable.ListBuffer
import java.lang.Math.toIntExact
import scala.reflect.ClassTag
import sys.process._
import geotrellis.raster._
import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.{CellType, DoubleArrayTile}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.input.PortableDataStream
import scala.collection.immutable.Map
import org.apache.commons.io.FilenameUtils;
import org.apache.spark.sql.functions.input_file_name
import scala.sys.process.Process
import sys.process._
import scala.Byte
import java.nio.file.{Files, Paths}
import java.nio.ByteBuffer
import java.util.Arrays 
import java.nio.ByteOrder
import java.lang.Math.toIntExact
import scala.reflect.ClassTag
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.HashPartitioner
import java.util.Calendar
import java.time.temporal.ChronoUnit
import org.apache.spark.Partitioner
import scala.io.Source

## Control

In [2]:
//CONTROL PARAMETERS (NDVI and EVI)
//Tilling 
var startRow = 1 
var endRow = 3000
var starColumn = 1
var endColumn = 6500
var numberTilesRow = 32
var numberTilesColumn = 32
var number_tiles = numberTilesRow * numberTilesColumn

//Settings files  parameters
var n_year = "19"
var n_points_per_year = "36"
val settigns_dir = "/data/local/home/parrot/minio2/settigs_dir/"
var settingsNameTemplate = "template.set"
val outputSettingName = "USA_part"
val settingsList: String = "/data/local/home/parrot/minio2/listfile_cluster.txt"
val executables_dir =  "/data/local/home/parrot/minio2/executables/"
val executable_path = executables_dir + "TSF_process.x64"

var filepath = "hdfs:///user/hadoop/evi_new_template.tif"

//Change here parameters for outpt paths
var results_folder = "performance_testing/"
var results_folderSOS = "performance_testing/"

var save_path = "/data/local/home/parrot/minio2/" + results_folder
var save_path_sos = "/data/local/home/parrot/minio2/" + results_folderSOS
var save_path_hdfs = "hdfs:///user/spark/" + results_folder



startRow = 1
endRow = 3000
starColumn = 1
endColumn = 6500
numberTilesRow = 32
numberTilesColumn = 32
number_tiles = 1024
n_year = 19
n_points_per_year = 36
settigns_dir = /data/local/home/parrot/minio2/settigs_dir/
settingsNameTemplate = template.set
outputSettingName = USA_part
settingsList = /data/local/home/parrot/minio2/listfile_cluster.txt
executables_dir = /data/local/home/parrot/minio2/executables/
executable_path = /data/local/home/parrot/minio2/executables/TSF_process.x64
results_folder = performance_testing/
results_folderSOS = performance_testing/
save_path = /data/local/home/parrot/minio2/performance_testing/
save_path_sos = /data/local/home/par...


/data/local/home/parrot/minio2/performance_testing/

## Create Settigns Files

In [3]:
def CreateSettigsFilesForEachNode() : Unit = {
    
    val rowRange = endRow - startRow
    var rowCounter = startRow
    
    var columnRange = endColumn - starColumn
    var columnCounter = starColumn

    var settingFile = new File("/data/local/home/parrot/minio2/" + settingsNameTemplate) 
//     val settingFile = new File("/data/local/home/parrot/minio2/" + settingsNameTemplate) 
    val fw_list = new FileWriter(settingsList)

    val lines = scala.io.Source.fromFile(settingFile).mkString.split("\\r?\\n")
    var endRowInc =startRow
    var endColumnInc =starColumn
    var tile_number = 1
    
    
    for( rowTile <- 1 to numberTilesRow){
      if( rowTile != numberTilesRow) {
        endRowInc = rowCounter + rowRange / numberTilesRow
      }
      else{
        // last task, compensate for the pixel increment and non-whole division
        endRowInc = (rowCounter + rowRange  / numberTilesRow) - numberTilesRow + 1 + (rowRange % numberTilesRow)
      }
        
      for( columnTile <- 1 to numberTilesColumn){
          if( columnTile != numberTilesColumn) {
            endColumnInc = columnCounter + columnRange / numberTilesColumn
          }
          else{
            // last task, compensate for the pixel increment and non-whole division
            endColumnInc = (columnCounter + columnRange  / numberTilesColumn) - numberTilesColumn + 1 + (columnRange % numberTilesColumn)
          }
        
          //change corresponding lines         
          lines(1)= "cluster" + tile_number +"      %Job_name (no blanks)" //change job name
          lines(10) = rowCounter.toString + " "+ endRowInc.toString + " "+ columnCounter.toString+" "+ endColumnInc.toString +"      %Processing window (start row end row start col end col)"
          lines(11) = n_year + " " + n_points_per_year  + "           %No. years and no. points per year"
          columnCounter +=  columnRange /numberTilesColumn +1

          println( "Settings file: " + tile_number )
          println(lines(10))

          val settingFileOut = new File(settigns_dir + outputSettingName+"_cluster" + ".set"+ tile_number) // Temporary File
          new PrintWriter(settingFileOut){write(lines.mkString("\n"))}.close() //close file
          fw_list.write(settigns_dir + outputSettingName+"_cluster" + ".set"+ tile_number + "\t" +"cluster" + tile_number +"_TS.tpa" + "\t" +"cluster" + tile_number +"_TS.ndx"  + "\n")
          tile_number = tile_number + 1
      }
        rowCounter +=  rowRange /numberTilesRow +1
        columnCounter = starColumn
        endColumnInc =starColumn
    }
     fw_list.close() 
}
    
CreateSettigsFilesForEachNode()

Settings file: 1
1 94 1 204      %Processing window (start row end row start col end col)
Settings file: 2
1 94 205 408      %Processing window (start row end row start col end col)
Settings file: 3
1 94 409 612      %Processing window (start row end row start col end col)
Settings file: 4
1 94 613 816      %Processing window (start row end row start col end col)
Settings file: 5
1 94 817 1020      %Processing window (start row end row start col end col)
Settings file: 6
1 94 1021 1224      %Processing window (start row end row start col end col)
Settings file: 7
1 94 1225 1428      %Processing window (start row end row start col end col)
Settings file: 8
1 94 1429 1632      %Processing window (start row end row start col end col)
Settings file: 9
1 94 1633 1836      %Processing window (start row end row start col end col)
Settings file: 10
1 94 1837 2040      %Processing window (start row end row start col end col)
Settings file: 11
1 94 2041 2244      %Processing window (start row en

Settings file: 87
189 282 4489 4692      %Processing window (start row end row start col end col)
Settings file: 88
189 282 4693 4896      %Processing window (start row end row start col end col)
Settings file: 89
189 282 4897 5100      %Processing window (start row end row start col end col)
Settings file: 90
189 282 5101 5304      %Processing window (start row end row start col end col)
Settings file: 91
189 282 5305 5508      %Processing window (start row end row start col end col)
Settings file: 92
189 282 5509 5712      %Processing window (start row end row start col end col)
Settings file: 93
189 282 5713 5916      %Processing window (start row end row start col end col)
Settings file: 94
189 282 5917 6120      %Processing window (start row end row start col end col)
Settings file: 95
189 282 6121 6324      %Processing window (start row end row start col end col)
Settings file: 96
189 282 6325 6500      %Processing window (start row end row start col end col)
Settings file: 97
28

Settings file: 171
471 564 2041 2244      %Processing window (start row end row start col end col)
Settings file: 172
471 564 2245 2448      %Processing window (start row end row start col end col)
Settings file: 173
471 564 2449 2652      %Processing window (start row end row start col end col)
Settings file: 174
471 564 2653 2856      %Processing window (start row end row start col end col)
Settings file: 175
471 564 2857 3060      %Processing window (start row end row start col end col)
Settings file: 176
471 564 3061 3264      %Processing window (start row end row start col end col)
Settings file: 177
471 564 3265 3468      %Processing window (start row end row start col end col)
Settings file: 178
471 564 3469 3672      %Processing window (start row end row start col end col)
Settings file: 179
471 564 3673 3876      %Processing window (start row end row start col end col)
Settings file: 180
471 564 3877 4080      %Processing window (start row end row start col end col)
Settings f

Settings file: 254
659 752 5917 6120      %Processing window (start row end row start col end col)
Settings file: 255
659 752 6121 6324      %Processing window (start row end row start col end col)
Settings file: 256
659 752 6325 6500      %Processing window (start row end row start col end col)
Settings file: 257
753 846 1 204      %Processing window (start row end row start col end col)
Settings file: 258
753 846 205 408      %Processing window (start row end row start col end col)
Settings file: 259
753 846 409 612      %Processing window (start row end row start col end col)
Settings file: 260
753 846 613 816      %Processing window (start row end row start col end col)
Settings file: 261
753 846 817 1020      %Processing window (start row end row start col end col)
Settings file: 262
753 846 1021 1224      %Processing window (start row end row start col end col)
Settings file: 263
753 846 1225 1428      %Processing window (start row end row start col end col)
Settings file: 264
75

Settings file: 337
941 1034 3265 3468      %Processing window (start row end row start col end col)
Settings file: 338
941 1034 3469 3672      %Processing window (start row end row start col end col)
Settings file: 339
941 1034 3673 3876      %Processing window (start row end row start col end col)
Settings file: 340
941 1034 3877 4080      %Processing window (start row end row start col end col)
Settings file: 341
941 1034 4081 4284      %Processing window (start row end row start col end col)
Settings file: 342
941 1034 4285 4488      %Processing window (start row end row start col end col)
Settings file: 343
941 1034 4489 4692      %Processing window (start row end row start col end col)
Settings file: 344
941 1034 4693 4896      %Processing window (start row end row start col end col)
Settings file: 345
941 1034 4897 5100      %Processing window (start row end row start col end col)
Settings file: 346
941 1034 5101 5304      %Processing window (start row end row start col end col)


...

Settings file: 747
2163 2256 2041 2244      %Processing window (start row end row start col end col)
Settings file: 748
2163 2256 2245 2448      %Processing window (start row end row start col end col)
Settings file: 749
2163 2256 2449 2652      %Processing window (start row end row start col end col)
Settings file: 750
2163 2256 2653 2856      %Processing window (start row end row start col end col)
Settings file: 751
2163 2256 2857 3060      %Processing window (start row end row start col end col)
Settings file: 752
2163 2256 3061 3264      %Processing window (start row end row start col end col)
Settings file: 753
2163 2256 3265 3468      %Processing window (start row end row start col end col)
Settings file: 754
2163 2256 3469 3672      %Processing window (start row end row start col end col)
Settings file: 755
2163 2256 3673 3876      %Processing window (start row end row start col end col)
Settings file: 756
2163 2256 3877 4080      %Processing window (start row end row start col

Settings file: 829
2351 2444 5713 5916      %Processing window (start row end row start col end col)
Settings file: 830
2351 2444 5917 6120      %Processing window (start row end row start col end col)
Settings file: 831
2351 2444 6121 6324      %Processing window (start row end row start col end col)
Settings file: 832
2351 2444 6325 6500      %Processing window (start row end row start col end col)
Settings file: 833
2445 2538 1 204      %Processing window (start row end row start col end col)
Settings file: 834
2445 2538 205 408      %Processing window (start row end row start col end col)
Settings file: 835
2445 2538 409 612      %Processing window (start row end row start col end col)
Settings file: 836
2445 2538 613 816      %Processing window (start row end row start col end col)
Settings file: 837
2445 2538 817 1020      %Processing window (start row end row start col end col)
Settings file: 838
2445 2538 1021 1224      %Processing window (start row end row start col end col)
S

Settings file: 911
2633 2726 2857 3060      %Processing window (start row end row start col end col)
Settings file: 912
2633 2726 3061 3264      %Processing window (start row end row start col end col)
Settings file: 913
2633 2726 3265 3468      %Processing window (start row end row start col end col)
Settings file: 914
2633 2726 3469 3672      %Processing window (start row end row start col end col)
Settings file: 915
2633 2726 3673 3876      %Processing window (start row end row start col end col)
Settings file: 916
2633 2726 3877 4080      %Processing window (start row end row start col end col)
Settings file: 917
2633 2726 4081 4284      %Processing window (start row end row start col end col)
Settings file: 918
2633 2726 4285 4488      %Processing window (start row end row start col end col)
Settings file: 919
2633 2726 4489 4692      %Processing window (start row end row start col end col)
Settings file: 920
2633 2726 4693 4896      %Processing window (start row end row start col

Settings file: 993
2915 3000 1 204      %Processing window (start row end row start col end col)
Settings file: 994
2915 3000 205 408      %Processing window (start row end row start col end col)
Settings file: 995
2915 3000 409 612      %Processing window (start row end row start col end col)
Settings file: 996
2915 3000 613 816      %Processing window (start row end row start col end col)
Settings file: 997
2915 3000 817 1020      %Processing window (start row end row start col end col)
Settings file: 998
2915 3000 1021 1224      %Processing window (start row end row start col end col)
Settings file: 999
2915 3000 1225 1428      %Processing window (start row end row start col end col)
Settings file: 1000
2915 3000 1429 1632      %Processing window (start row end row start col end col)
Settings file: 1001
2915 3000 1633 1836      %Processing window (start row end row start col end col)
Settings file: 1002
2915 3000 1837 2040      %Processing window (start row end row start col end col

CreateSettigsFilesForEachNode: ()Unit


## Prepare Jobs in RDD

In [3]:
class ExactPartitioner(override val numPartitions: Int) extends Partitioner {

  def getPartition(key: Any): Int = key match {
    // `k` is assumed to go continuously from 0 to elements-1.
    case _ =>      key.asInstanceOf[Int] 
  }
}


defined class ExactPartitioner


In [4]:

//Argumenets: settings path, output .tpa file
var settings_rdd : RDD[(String, String,String)] = sc.textFile("file://" + settingsList)
.map(str => str.split("\t"))
.map(arr => (arr(0), arr(1), arr(2)))

// Repartition on each line 
//default partitioner can assign several rdd records (block_ids) to one partitioner, few empty
var settings_rdd_part = settings_rdd.repartition(number_tiles).zipWithIndex.map{case (k,v) => (v.toInt,k)} 

var one_task_per_record : RDD[(String,String,String)]= settings_rdd_part.partitionBy(new ExactPartitioner(number_tiles)).values


one_task_per_record.mapPartitionsWithIndex((idx, it) => Iterator((idx, it.toList))).toDF("partition_id", "block_ids").show(2024,false)



Waiting for a Spark session to start...

[Stage 19:===================================================> (388 + 11) / 399]+------------+-------------------------------------------------------------------------------------------------------------+
|partition_id|block_ids                                                                                                    |
+------------+-------------------------------------------------------------------------------------------------------------+
|0           |[[/data/local/home/parrot/minio2/settigs_dir/USA_part_cluster.set276,cluster276_TS.tpa,cluster276_TS.ndx]]   |
|1           |[[/data/local/home/parrot/minio2/settigs_dir/USA_part_cluster.set790,cluster790_TS.tpa,cluster790_TS.ndx]]   |
|2           |[[/data/local/home/parrot/minio2/settigs_dir/USA_part_cluster.set277,cluster277_TS.tpa,cluster277_TS.ndx]]   |
|3           |[[/data/local/home/parrot/minio2/settigs_dir/USA_part_cluster.set791,cluster791_TS.tpa,cluster791_TS.ndx]]   |
|4           |[[/data/local/home/parrot/minio

|1022        |[[/data/local/home/parrot/minio2/settigs_dir/USA_part_cluster.set275,cluster275_TS.tpa,cluster275_TS.ndx]]   |
|1023        |[[/data/local/home/parrot/minio2/settigs_dir/USA_part_cluster.set789,cluster789_TS.tpa,cluster789_TS.ndx]]   |
+------------+-------------------------------------------------------------------------------------------------------------+



settings_rdd = MapPartitionsRDD[3] at map at <console>:95
settings_rdd_part = MapPartitionsRDD[9] at map at <console>:101
one_task_per_record = MapPartitionsRDD[11] at values at <console>:103


MapPartitionsRDD[11] at values at <console>:103

## Execute !

In [ ]:
val res = one_task_per_record.map( setting_tile => { Seq(executable_path, setting_tile._1,"1" ).!
                               Seq("mv",setting_tile._2,save_path).!
                               Seq("mv",setting_tile._3,save_path).!})
res.collect()


## Only Run if Execution Problems, Will stop the TimeSat process from within the machines

In [ ]:
var kill = one_task_per_record.map( setting_tile => { Seq("sudo","pkill","TSF_process.x64").!})
kill.collect()

In [7]:

//Execute to set the permissions: HADOOP_USER_NAME=root hadoop fs -chmod -R 777 /user/spark
val cmd = "hadoop fs -copyFromLocal " + save_path + " " + "/user/spark"
Process(cmd)!

cmd = hadoop fs -copyFromLocal /data/local/home/parrot/minio2/NDVI_experiment_11/ /user/spark


0

In [114]:
var data = sc.textFile("file:///data/local/home/parrot/minio2/listfileNdviNew.txt")
var count  = names.first()
var filtered_dates = data.filter(row => row != count)
var new2 = filtered_dates.map( data => data.split("\\.")).map(path => path(0).split("/")).map(path => path(path.size -1))

var new3 =new2.zipWithIndex().map { case (v, i) => (i+1, v) } //cache
var index_date  = new3.collectAsMap().map(kv => (kv._1,kv._2)).toMap

var index_dateBC = sc.broadcast(index_date)

index_date.get(28)


data = file:///data/local/home/parrot/minio2/listfileNdviNew.txt MapPartitionsRDD[459] at textFile at <console>:266
count = 684
filtered_dates = MapPartitionsRDD[460] at filter at <console>:268
new2 = MapPartitionsRDD[463] at map at <console>:269
new3 = MapPartitionsRDD[465] at map at <console>:271
index_date = Map(645 -> 20161123, 69 -> 20001123, 629 -> 20160613, 365 -> 20090213, 138 -> 20021024, 101 -> 20011013, 479 -> 20120413, 347 -> 20080813, 333 -> 20080324, 628 -> 20160603, 249 -> 20051123, 518 -> 20130513, 468 -> 20111224, 234 -> 20050623, 555 -> 20140524, 666 -> 20170623, 88 -> 200106...


Map(645 -> 20161123, 69 -> 20001123, 629 -> 20160613, 365 -> 20090213, 138 -> 20021024, 101 -> 20011013, 479 -> 20120413, 347 -> 20080813, 333 -> 20080324, 628 -> 20160603, 249 -> 20051123, 518 -> 20130513, 468 -> 20111224, 234 -> 20050623, 555 -> 20140524, 666 -> 20170623, 88 -> 20010603, 481 -> 20120503, 352 -> 20081003, 408 -> 20100423, 170 -> 20030913, 523 -> 20130703, 582 -> 20150221, 115 -> 20020303, 683 -> 20171213, 217 -> 20050103, 276 -> 20060824, 308 -> 20070713, 5 -> 19990213, 449 -> 20110613, 120 -> 20020423, 247 -> 20051103, 379 -> 20090703, 440 -> 20110313, 655 -> 20170303, 511 -> 20130303, 614 -> 20160113, 269 -> 20060613, 677 -> 20171013, 202 -> 20040803, 597 -> 20150724, 10 -> 19990403, 385 -> 20090903, 384 -> 20090824, 56 -> 20000713, 533 -> 20131013, 550 -> 20140403, 142 -> 20021203, 500 -> 20121113, 472 -> 20120203, 340 -> 20080603, 538 -> 20131203, 153 -> 20030324, 670 -> 20170803, 174 -> 20031024, 404 -> 20100313, 185 -> 20040213, 42 -> 20000222, 417 -> 20100724, 

## Read Binary Input into rdd

In [3]:

var rdd_tpa = sc.binaryFiles(save_path_hdfs + "/*.tpa").map(m => (FilenameUtils.getBaseName(m._1).split("_")(0), m._2))
var rdd_tpx = sc.binaryFiles(save_path_hdfs+ "/*.ndx").map(m => (FilenameUtils.getBaseName(m._1).split("_")(0), m._2))

var all :RDD[(String,(PortableDataStream,PortableDataStream))]  = rdd_tpa.join(rdd_tpx)



Waiting for a Spark session to start...

rdd_tpa = MapPartitionsRDD[1] at map at <console>:88
rdd_tpx = MapPartitionsRDD[3] at map at <console>:89
all = MapPartitionsRDD[6] at join at <console>:91


MapPartitionsRDD[6] at join at <console>:91

## Extract Seasonality Information per pixel

In [4]:

def getSeasons(filename: String, indexStream: PortableDataStream, seasonalityStream: PortableDataStream  ): Array[Float] = {
    var index : Array[Byte] = indexStream.toArray()
    if(index.length != 0){
        var seasonality : Array[Byte] = seasonalityStream.toArray()
    //     var index_tuple : Array[Long] = new Array[Long]( index.length/8)
        var nseasons_int_def = toIntExact(ByteBuffer.wrap(index.slice(16,24)).order(ByteOrder.LITTLE_ENDIAN).getLong)
        var array = new Array[Float](index.length/32 * nseasons_int_def *16)
        var arrayIndex = 0

        //loop on the pixel positions 
        for(i <- 0 until index.length by 32){ 

            var nseasons_int = toIntExact(ByteBuffer.wrap(index.slice(i+16,i+24)).order(ByteOrder.LITTLE_ENDIAN).getLong)   // number seasons
            var position_int = toIntExact(ByteBuffer.wrap(index.slice(i+24,i+32)).order(ByteOrder.LITTLE_ENDIAN).getLong) -1 //adjust for somewhat -1 offset in the file 

            //loop on the available seasons 
            for (z <- 1 to nseasons_int){
                array(arrayIndex) = ByteBuffer.wrap(index.slice(i,i+8)).order(ByteOrder.LITTLE_ENDIAN).getLong //row
                array(arrayIndex + 1) = ByteBuffer.wrap(index.slice(i+8,i+16)).order(ByteOrder.LITTLE_ENDIAN).getLong //column
                array(arrayIndex + 2) = z //number of season we are reading
                if (z.equals(1)) position_int+= 12  //skip row,column, number seasons, we have them from index file
                //loap the  seasonality parametars
                for (m <- 1 to 13){ 
                    array(arrayIndex + 2 + m) = (ByteBuffer.wrap(seasonality.slice(position_int, position_int+4)).order(ByteOrder.LITTLE_ENDIAN).getFloat)
                    position_int+= 4
                }
                arrayIndex = arrayIndex + 3 + 13
            }
        }
        return array
      }
    else{
        return new Array[Float](16).fill(-1)
    }
}


var all_seasonality_parametars  = all.flatMap(m => { getSeasons(m._1, m._2._2, m._2._1).grouped(16).toList}).filter(m => m(0) != -1)

//here we drop the othe seasonality parameteers and get SOS only 
//tuple in the form (x,y,season,SOS) --> [(1609.0,2721.0,1.0,45.88774), (1609.0,2721.0,2.0,80.46434), ...]
var sos_tuple = all_seasonality_parametars.map(m => (m(0),m(1),m(2),m(3)))



all_seasonality_parametars = MapPartitionsRDD[8] at filter at <console>:129
sos_tuple = MapPartitionsRDD[9] at map at <console>:133


getSeasons: (filename: String, indexStream: org.apache.spark.input.PortableDataStream, seasonalityStream: org.apache.spark.input.PortableDataStream)Array[Float]


MapPartitionsRDD[9] at map at <console>:133

In [5]:
var sos_tuple_s1  = sos_tuple.filter(x => x._3 == 15.toFloat).filter(x => x._1 == 1000).map{case (x,y,season,value) => {
       
        (x,y,season,value)
    }}

sos_tuple_s1.take(1000).foreach(println)

[Stage 14:=====================================================>(498 + 2) / 500](1000.0,96.0,15.0,514.1116)
(1000.0,97.0,15.0,514.4568)
(1000.0,98.0,15.0,514.251)
(1000.0,99.0,15.0,514.2909)
(1000.0,100.0,15.0,513.82007)
(1000.0,101.0,15.0,513.6108)
(1000.0,102.0,15.0,512.8946)
(1000.0,103.0,15.0,512.8385)
(1000.0,104.0,15.0,512.2126)
(1000.0,105.0,15.0,548.6117)
(1000.0,106.0,15.0,548.8082)
(1000.0,107.0,15.0,548.7389)
(1000.0,108.0,15.0,549.1417)
(1000.0,109.0,15.0,549.12225)
(1000.0,110.0,15.0,548.68555)
(1000.0,111.0,15.0,548.2029)
(1000.0,112.0,15.0,547.9896)
(1000.0,113.0,15.0,548.7329)
(1000.0,114.0,15.0,550.7386)
(1000.0,115.0,15.0,513.9149)
(1000.0,116.0,15.0,514.2395)
(1000.0,117.0,15.0,550.74927)
(1000.0,118.0,15.0,512.66156)
(1000.0,119.0,15.0,512.2675)
(1000.0,120.0,15.0,512.3832)
(1000.0,121.0,15.0,512.3692)
(1000.0,122.0,15.0,512.03937)
(1000.0,123.0,15.0,546.22894)
(1000.0,124.0,15.0,513.1123)
(1000.0,125.0,15.0,513.1883)
(1000.0,126.0,15.0,512.71875)
(1000.0,127.0,15.0

(1000.0,774.0,15.0,511.40918)
(1000.0,775.0,15.0,511.24557)
(1000.0,776.0,15.0,548.50916)
(1000.0,777.0,15.0,509.47794)
(1000.0,778.0,15.0,549.0085)
(1000.0,779.0,15.0,547.8892)
(1000.0,780.0,15.0,546.11896)
(1000.0,781.0,15.0,546.86566)
(1000.0,782.0,15.0,546.72516)
(1000.0,783.0,15.0,546.0955)
(1000.0,784.0,15.0,546.1761)
(1000.0,785.0,15.0,545.60956)
(1000.0,786.0,15.0,547.2512)
(1000.0,787.0,15.0,513.65)
(1000.0,788.0,15.0,512.8974)
(1000.0,789.0,15.0,513.15436)
(1000.0,790.0,15.0,511.51086)
(1000.0,791.0,15.0,511.31677)
(1000.0,792.0,15.0,510.8606)
(1000.0,793.0,15.0,513.73)
(1000.0,794.0,15.0,513.04987)
(1000.0,795.0,15.0,512.9492)
(1000.0,796.0,15.0,512.987)
(1000.0,797.0,15.0,511.48016)
(1000.0,798.0,15.0,512.5106)
(1000.0,799.0,15.0,513.14874)
(1000.0,800.0,15.0,514.01)
(1000.0,801.0,15.0,549.9288)
(1000.0,802.0,15.0,549.3686)
(1000.0,803.0,15.0,547.42456)
(1000.0,804.0,15.0,546.7192)
(1000.0,805.0,15.0,546.5734)
(1000.0,806.0,15.0,546.6554)
(1000.0,807.0,15.0,547.2843)
(1000.

(1000.0,1821.0,15.0,514.33734)
(1000.0,1822.0,15.0,514.5406)
(1000.0,1823.0,15.0,514.1411)
(1000.0,1825.0,15.0,514.2434)
(1000.0,1826.0,15.0,513.92285)
(1000.0,1828.0,15.0,513.6562)
(1000.0,2141.0,15.0,516.38715)
(1000.0,2142.0,15.0,515.69574)
(1000.0,2143.0,15.0,516.83136)
(1000.0,2146.0,15.0,516.82043)
(1000.0,2147.0,15.0,516.9239)
(1000.0,2149.0,15.0,516.90186)
(1000.0,2221.0,15.0,516.98517)
(1000.0,2222.0,15.0,516.83)
(1000.0,2223.0,15.0,517.06494)
(1000.0,2224.0,15.0,517.21545)
(1000.0,2225.0,15.0,517.24066)
(1000.0,2226.0,15.0,517.13135)
(1000.0,2227.0,15.0,517.05817)
(1000.0,2228.0,15.0,516.66406)
(1000.0,2229.0,15.0,516.791)
(1000.0,2230.0,15.0,516.7246)
(1000.0,2231.0,15.0,516.57526)
(1000.0,2232.0,15.0,516.74146)
(1000.0,2233.0,15.0,516.7498)
(1000.0,2234.0,15.0,516.5536)
(1000.0,2235.0,15.0,516.512)
(1000.0,2236.0,15.0,516.6029)
(1000.0,2237.0,15.0,516.50226)
(1000.0,2238.0,15.0,516.6443)
(1000.0,2239.0,15.0,516.6496)
(1000.0,2240.0,15.0,516.67535)
(1000.0,2241.0,15.0,516.46

sos_tuple_s1 = MapPartitionsRDD[12] at map at <console>:96


MapPartitionsRDD[12] at map at <console>:96

In [5]:
//find number of seasons //think of a better way, instead of going throug all the rdd
val n_seasons_found : Int = sos_tuple.map(x => x._3).max().toInt

[Stage 2:=====================================================>(998 + 2) / 1000]

n_seasons_found = 18


18

## Prepare Template

In [6]:
//Single band GeoTiff
//Since it is a single GeoTiff, it will be a RDD with a tile.
var geoTiffs : RDD[(ProjectedExtent, Tile)] = sc.hadoopGeoTiffRDD(filepath)
var proExtents_RDD : RDD[ProjectedExtent] = geoTiffs.keys
var tiles_RDD : RDD[Tile]= geoTiffs.values


var grids_RDD = tiles_RDD.map(m => m.toArrayDouble())

var bands_withIndex = grids_RDD.zipWithIndex().map { case (v, i) => (i, v) }

//Array[(Long, Double)]
var grid0 = bands_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}    //

var extents_withIndex = proExtents_RDD.zipWithIndex().map{case (e,v) => (v,e)} //cache
var projected_extent = (extents_withIndex.filter(m => m._1 == 0).values.collect())(0) //.cache.collect())(0)


filepath = hdfs:///user/hadoop/evi_new_template.tif
geoTiffs = MapPartitionsRDD[12] at mapPartitions at HadoopGeoTiffRDD.scala:109
proExtents_RDD = MapPartitionsRDD[13] at keys at <console>:95
tiles_RDD = MapPartitionsRDD[14] at values at <console>:96
grids_RDD = MapPartitionsRDD[15] at map at <console>:99
bands_withIndex = MapPartitionsRDD[17] at map at <console>:101
grid0 = MapPartitionsRDD[22] at map at <console>:104


extents_withIndex: org.apache.spark...


MapPartitionsRDD[22] at map at <console>:104

## Align Seasons

In [7]:
var filtered = sos_tuple.filter(x => x._3 == 1.0 && x._4 > 27).map{ case (x, y,season,value) => (x, y) }

//NDVI
var filtered2 = sos_tuple.filter(x => x._3 == 18.0 && (x._4 > 639 || x._4 == 0)).map{ case (x, y,season,value) => (x, y) }
var intersection = sc.broadcast(filtered.intersection(filtered2).collect.toSet)

var aligned_seasons_1 = sos_tuple.map{ case (x, y,season,value) => {
    if (intersection.value.contains((x,y))){
        (x, y,season + 1,value)
    }else{
    (x, y,season,value)
    }
    
}} //cache


[Stage 8:==================================================>  (947 + 49) / 1000]

filtered = MapPartitionsRDD[28] at map at <console>:96
filtered2 = MapPartitionsRDD[30] at map at <console>:99
intersection = Broadcast(10)
aligned_seasons_1 = MapPartitionsRDD[37] at map at <console>:108


MapPartitionsRDD[37] at map at <console>:108

## Generate GeoTiff for each available season

In [8]:


var num_cols :Int = 6500
var num_rows :Int = 3000
var num_colsB = sc.broadcast(num_cols)
var start_year = 1999
var start_yearB = sc.broadcast(start_year)
var season_number = 1




for( season_number <- 1 to n_seasons_found + 1){
     
    var sos_tuple_s1  = aligned_seasons_1.filter(x => x._3 == season_number.toFloat).filter(x => x._4 != 0.0).map{case (x,y,season,value) => {
        var rounded = Math.rint( (value -1)*10.139)   //round to int by using 2 digits of precision
        var date_TimeSat : Calendar = Calendar.getInstance()
        
        var leap_year_days = 0
        if(season_number >= 2 && season_number<=5){
            leap_year_days = 1
        }
        else if (season_number >= 6 && season_number<= 9){
             leap_year_days = 2
        }
        else if (season_number >= 10 && season_number<= 13){
             leap_year_days = 3
        }
        else if (season_number >= 14 && season_number<= 17){
             leap_year_days = 4
        }else if (season_number> 17){
            leap_year_days = 5
        }   
        
        date_TimeSat.set(start_yearB.value , 0, 3)
        date_TimeSat.add(Calendar.DATE, rounded.toInt + leap_year_days)
        var day_of_year = date_TimeSat.get(Calendar.DAY_OF_YEAR)
        //offster early spings
        if(day_of_year < 270){
            (x,y,season,day_of_year)}
        else{
            (x,y,season,day_of_year- 365)}
        
    
    }}
    
    print("=====Calculating Season "+ season_number)
//      sos_tuple_s2.take(1000).foreach(println)

    //The .get(col,row) methods are implemented by using the equation cols * row + col to translate from grid coordinates to array index
    var season_to_array = sos_tuple_s1.map(x => (num_colsB.value.toInt*(x._1.toInt -1) + (x._2.toInt- 1).toLong, x._4))

    var result_2: RDD[(Long, (Double, Option[Int]))] = grid0.leftOuterJoin(season_to_array)
    // result_2.take(20).foreach(println)

    //TODO: 1. If value in original .tiff  , but None in seasonality ==> set NaN for the corresponding index
    var result_3: RDD[(Long, Double)] = result_2.map { 
      case (index, (value_org, value_pheno)) => {if (value_pheno.isDefined)  (index,value_pheno.get)
                                                 else (index, Double.NaN)  }
    }
    

    var season = DoubleArrayTile(result_3.sortBy(_._1).map(_._2).collect() , num_cols, num_rows)


    var geoTif = new SinglebandGeoTiff(season, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
    var year = start_year + season_number -1
    var output = save_path_sos + year + ".tif"
    GeoTiffWriter.write(geoTif, output)    
 }



[Stage 217:==================================================>(988 + 12) / 1000]=====Calculating Season 2=====Calculating Season 3=====Calculating Season 4=====Calculating Season 51000]=====Calculating Season 6=====Calculating Season 7=====Calculating Season 8=====Calculating Season 9=====Calculating Season 10=====Calculating Season 11=====Calculating Season 12=====Calculating Season 13=====Calculating Season 14=====Calculating Season 15=====Calculating Season 16=====Calculating Season 17=====Calculating Season 18=====Calculating Season 19

num_cols = 6500
num_rows = 3000
num_colsB = Broadcast(11)
start_year = 1999
start_yearB = Broadcast(12)
season_number = 1


1